### Worst use of Window Functions

In [84]:
# Create Spark Session

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Worst use of Window Function") \
    .master("local[*]") \
    .getOrCreate()

spark

In [85]:
# Set Spark default config
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [86]:
# Read example data set
from pyspark.sql.functions import sum, expr, monotonically_increasing_id

df = spark.read.format("csv").option("header", True).load("dataset/sales.csv")

print("Initial Partition after read: " + str(df.rdd.getNumPartitions()))

Initial Partition after read: 8


In [87]:
# Use row_number() to generate ids
df_with_row_num = df.withColumn("_id", expr("row_number() over (order by null)"))

# Write the dataset in noop for performance benchmarking
df_with_row_num.write.format("noop").mode("overwrite").save()

In [78]:
# Validate the number of partitions
df_with_row_num.rdd.getNumPartitions()

1

In [88]:
# Use Spark in-built monotonically_increasing_id to generate ids
df_with_incr_id = df.withColumn("_id", monotonically_increasing_id())

# Write the dataset in noop for performance benchmarking
df_with_incr_id.write.format("noop").mode("overwrite").save()

In [80]:
# Validate the number of partitions
df_with_incr_id.rdd.getNumPartitions()

8

In [83]:
spark.stop()